## **D3TOP - Tópicos em Ciência de Dados (IFSP Campinas)**
**Vinícius Vieira Albano (CP3013677)** <br/>

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

<hr/>

# **Aplicação de Processamento de Linguagem Natural e Clustering para Simplificação da Legislação Municipal: Um Estudo de Caso de Sorocaba**

## **Descrição e motivação do problema:**

A legislação brasileira é notoriamente vasta e complexa, tornando o trabalho de simplificação legal lento e desafiador. Em particular, leis municipais, como as de nomeações de logradouros ou de concessão de títulos honoríficos, em cidades como Sorocaba, podem ser unificadas ou revogadas para tornar a legislação mais eficiente e acessível. A motivação para este trabalho é criar um modelo de Processamento de Linguagem Natural (PLN) e clustering que possa auxiliar na identificação dessas leis e contribuir para a eficiência do trabalho legislativo.

## **Descrição da base de dados (e do processo de obtenção dos dados, se for o caso):**

Os dados para este trabalho são obtidos do site "Leis Municipais", que reúne a legislação municipal de várias cidades brasileiras, incluindo Sorocaba. O processo de obtenção de dados envolve a raspagem de dados (webscraping) dessas leis, usando um aplicativo chamado ScrapeStorm e bibliotecas como BeautifulSoup. Os dados são então armazenados em um formato adequado para análise posterior, como um DataFrame do pandas.

Uma dificuldade enfrentada é superar os CAPTCHAs, que aparecem frequentemente no site. Desta forma, precisamos fazer retentativas até conseguir completar a base de dados.

## **Objetivo de negócio ou científico associado ao problema:**
O objetivo deste trabalho é desenvolver um modelo eficiente de PLN e clustering que possa identificar leis municipais passíveis de unificação ou revogação. Isso pode levar a um sistema mais simplificado e eficiente de legislação municipal, beneficiando não apenas assessores legislativos, mas também os cidadãos. Do ponto de vista científico, este trabalho também pode contribuir para o campo do PLN, fornecendo uma aplicação real e valiosa dessa tecnologia.

# **Pré-requisitos**

In [3]:
# Instalar dependências do python
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for gensim from https://files.pythonhosted.org/packages/14/34/f1e056feda95330f7d8beef6771e3441ce0e8e2d1f55bf754b0b0594b234/gensim-4.3.2-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 10.3 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 9.0 MB/s eta 0:00:0000:0100:01m
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 5.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for wordcloud: filename=wordcloud-1.9.2-cp311-cp311-linux_aarch64.whl size=178048 sha256=e25446f13f59a1d

In [4]:
# Importar bibliotecas necessárias para este projeto
from bs4 import BeautifulSoup, Comment
from datetime import date
from IPython.core.display import display, HTML
from itertools import chain
import concurrent.futures
import json
import pandas as pd
import re
import requests

/tmp/ipykernel_71559/2233280330.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# **Obtenção dos dados**

Começamos o trabalho acessando o site "Leis Municipais" e fazendo uma busca por todas as leis ordinárias e complementares. A busca retorna um índice paginado com todas as leis, ordenado da mais recente para a mais antiga.

A página inicial do índice possui esta URL: `https://leismunicipais.com.br/prefeitura/sp/sorocaba?o=&q=&types=28&types=4`

Ao avançarmos para a próxima página, um parâmetro `&page=2` foi adicionado à URL.

In [ ]:
# Acessar a página inicial, que nos trará informações sobre os links das legislações municipais
page = requests.get('https://leismunicipais.com.br/prefeitura/sp/sorocaba?o=&q=&types=28&types=4')
site = BeautifulSoup(page.content, 'html.parser')

In [ ]:
# Obter a data de hoje, somente para documentação
today = date.today().strftime("%d/%m/%Y")

In [ ]:
# Encontrar o link da última página de índice de leis
last_page_link = site.find('a', attrs={'title': 'Última página'}).get('href')

In [ ]:
# Extrair o número da última página de índice de leis
last_page_number = int(re.search('&page=(\d+)', last_page_link).groups()[0])
print(f'O site possui atualmente ({today}) {last_page_number} páginas de legislação ordinária e complementar')

O site possui atualmente (03/06/2023) 1282 páginas de legislação ordinária e complementar


Descobrimos quantas páginas de legislação municipal de Sorocaba existem. Agora, precisaremos acessar uma a uma, para obter os links de cada legislação e, posteriormente acessá-las, individualmente.

Analisando a estrutura da página, verificamos que elas estão em elementos HTML com a classe `.law-link`. Vamos extraí-los para verificação:

In [ ]:
# Na classe .law-link, temos as URLs individuais das legislações
legislation_links = [element.get('href') for element in site.select('.law-link')]
print(f'Existem {len(legislation_links)} links de legislação em cada página')

Existem 10 links de legislação em cada página


In [ ]:
# Visualizar links das leis
for link in legislation_links:
  print(link)

/a1/sp/s/sorocaba/lei-ordinaria/2023/1282/12814/lei-ordinaria-n-12814-2023-dispoe-sobre-denominacao-de-jose-antonio-pascoto-a-uma-via-publica
/a1/sp/s/sorocaba/lei-ordinaria/2023/1282/12813/lei-ordinaria-n-12813-2023-dispoe-sobre-denominacao-de-vitor-hage-a-uma-via-publica-e-da-outras-providencias
/a1/sp/s/sorocaba/lei-ordinaria/2023/1282/12812/lei-ordinaria-n-12812-2023-dispoe-sobre-denominacao-de-lisardo-cunha-dias-a-uma-via-publica-e-da-outras-providencias
/a1/sp/s/sorocaba/lei-ordinaria/2023/1282/12811/lei-ordinaria-n-12811-2023-dispoe-sobre-denominacao-de-desirre-ferraz-cardoso-a-uma-via-publica-de-nossa-cidade-e-da-outras-providencias
/a1/sp/s/sorocaba/lei-ordinaria/2023/1281/12810/lei-ordinaria-n-12810-2023-institui-no-calendario-oficial-do-municipio-o-dia-do-sociologo-e-da-outras-providencias
/a1/sp/s/sorocaba/lei-ordinaria/2023/1281/12809/lei-ordinaria-n-12809-2023-altera-a-redacao-dos-artigos-7-8-9-e-10-da-lei-municipal-n-11982-de-14-de-maio-de-2019-e-da-outras-providencias
/

Agora, vamos salvar os links de todas as páginas

In [ ]:
# Função para fazer a requisição e extração de dados
def fetch(url):
    data = []
    try:
      response = requests.get(url)
      response.raise_for_status()  # Lança uma exceção se a requisição falhar
      soup = BeautifulSoup(response.content, 'html.parser')
      for element in soup.select('.law-link'):
          legislation_link = 'https://leismunicipais.com.br' + element.get('href')
          title = element.select_one('.title strong').get_text()
          data.append({'Title': title, 'Title_link': legislation_link})
    except requests.HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')
    return data

In [ ]:
%%time
# Lista de URLs
urls = ['https://leismunicipais.com.br/prefeitura/sp/sorocaba?o=&q=&types=28&types=4&page={}'.format(i) for i in range(1, last_page_number + 1)]

# Limitar o número de requisições simultâneas para evitar problemas de bloqueio no site
max_workers = 10

# Fazer as requisições e extrações de dados em paralelo
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    all_legislation_data = list(executor.map(fetch, urls))

# Achatando a lista de listas em uma única lista
all_legislation_data = list(chain.from_iterable(all_legislation_data))

CPU times: user 1min 20s, sys: 1.59 s, total: 1min 22s
Wall time: 2min 8s


In [ ]:
# Salvar em um dataframe
df_index = pd.DataFrame(all_legislation_data)

In [ ]:
# Verificar a quantidade de leis a serem salvas
len(df_index)

12812

In [ ]:
# Observar as primeiras linhas do dataset
df_index.head()

,Title,Title_link
0,Lei Ordinária 12814/2023,https://leismunicipais.com.br/a1/sp/s/sorocaba...
1,Lei Ordinária 12813/2023,https://leismunicipais.com.br/a1/sp/s/sorocaba...
2,Lei Ordinária 12812/2023,https://leismunicipais.com.br/a1/sp/s/sorocaba...
3,Lei Ordinária 12811/2023,https://leismunicipais.com.br/a1/sp/s/sorocaba...
4,Lei Ordinária 12810/2023,https://leismunicipais.com.br/a1/sp/s/sorocaba...


In [ ]:
# Salvar o índice completo de leis em CSV, para utilizarmos depois
df_index.to_csv('data/Índice de Leis de Sorocaba.csv', index=False)

Para acessar as leis individualmente, vamos utilizar o aplicativo ScrapeStorm. Isso se deve ao fato de o site "Leis Municipais" utilizar Javascript para carregar os textos. Assim, precisamos de uma forma de aguardar o processamento antes de obter os dados.

Além disso, eventualmente a página retorna um CAPTCHA. Precisamos de uma forma de detectar quando ele apareceu, para resolvê-lo, e continuar o processo de extração automatizada das leis.

Analizando a estrutura da página, percebemos que o conteúdo das leis está dentro da tag HTML com classe `.law-content`, que possui algumas subdivisões em HTML para formatação e que podem ser úteis para extrair contexto. Assim, ao extrair o conteúdo do site, salvamos o título da página, a URL e o conteúdo HTML dessa tag `.law-content`.

O aplicativo ScrapeStorm só permite exportar 100 linhas de arquivo por dia no plano gratuito. Assim, assinamos um mês do plano profissional, por US$ 50, que permite exportar 10.000 linhas por dia. Existem outros planos com limite superior, mas não justificam o investimento agora.

Desta forma, após cerca de 48 horas de trabalho, conseguimos salvar (parcialmente) as quase 13 mil leis de Sorocaba e exportar em dois arquivos CSV, dado o limite de 10 mil linhas por dia.

Conseguimos salvar parcialmente, pois em diversos momentos o CAPTCHA impediu salvar o conteúdo das leis e tivemos que pausar a coleta, resolvê-lo e retomar. Ainda assim, a passagem automática do algoritmo acabava pulando diversas páginas quando o CAPTCHA aparecia, pelo fato de não conseguir extrair o conteúdo. Desta forma, precisamos filtrar nossos CSVs e verificar quais leis ficaram faltando e rodar novamente no ScrapeStorm.

In [ ]:
# Carregar os datasets iniciais
df1 = pd.read_csv('data/Leis de Sorocaba SP (Parcial 1).csv')
df2 = pd.read_csv('data/Leis de Sorocaba SP (Parcial 2).csv')

In [ ]:
# Concatenar os dois dataframes
df = pd.concat([df1, df2], ignore_index=True)

In [ ]:
# Ordenar o dataframe de modo que linhas não vazias de 'Law HTML' venham antes das vazias
df_sorted = df.sort_values(by='Law HTML', na_position='last')

# Remover duplicatas baseado na coluna 'Title_link', mantendo a primeira (não vazia, se houver)
df_unique = df_sorted.drop_duplicates(subset='Title_link', keep='first')

In [ ]:
df_unique.head()

,Title,Title_link,Law HTML
177,Lei Ordinária 12632 2022 de Sorocaba SP,https://leismunicipais.com.br/a1/sp/s/sorocaba...,<!-- BEGIN ANCORA -->\n ...
157,Lei Ordinária 12639 2022 de Sorocaba SP,https://leismunicipais.com.br/a1/sp/s/sorocaba...,<!-- BEGIN ANCORA -->\n ...
175,Lei Ordinária 12634 2022 de Sorocaba SP,https://leismunicipais.com.br/a1/sp/s/sorocaba...,<!-- BEGIN ANCORA -->\n ...
173,Lei Ordinária 12636 2022 de Sorocaba SP,https://leismunicipais.com.br/a1/sp/s/sorocaba...,<!-- BEGIN ANCORA -->\n ...
174,Lei Ordinária 12635 2022 de Sorocaba SP,https://leismunicipais.com.br/a1/sp/s/sorocaba...,<!-- BEGIN ANCORA -->\n ...


In [ ]:
# Unir ao índice, para ver se não há leis que ficaram faltando de serem buscadas
df_index_with_content = pd.merge(df_index, df_unique, on=['Title_link'], how='left')

# Agora, vamos verificar quais linhas têm valores ausentes após o merge
missing_rows = df_index_with_content[df_index_with_content['Law HTML'].isnull() | (df_index_with_content['Law HTML'] == '')]
print(f"Há {len(missing_rows)} valores faltantes na coluna 'Law HTML'.")

Há 2613 valores faltantes na coluna 'Law HTML'.


In [ ]:
missing_rows.head()

,Title_x,Title_link,Title_y,Law HTML
0,Lei Ordinária 12814/2023,https://leismunicipais.com.br/a1/sp/s/sorocaba...,NaN,NaN
1,Lei Ordinária 12813/2023,https://leismunicipais.com.br/a1/sp/s/sorocaba...,NaN,NaN
2,Lei Ordinária 12812/2023,https://leismunicipais.com.br/a1/sp/s/sorocaba...,NaN,NaN
3,Lei Ordinária 12811/2023,https://leismunicipais.com.br/a1/sp/s/sorocaba...,NaN,NaN
4,Lei Ordinária 12810/2023,https://leismunicipais.com.br/a1/sp/s/sorocaba...,NaN,NaN


In [ ]:
# Selecionar as URLs das leis que faltam o campo 'Law HTML'
missing_links = missing_rows['Title_link']

missing_links.to_csv('data/missing_links.csv', index=False)

Após obter quais leis permaneciam faltando, rodamos o ScrapeStorm novamente e salvamos o que faltava, em algumas tentativas. Repetimos o processo acima diversas vezes até garantir que não havia mais nenhuma lei faltante. Para encurtar o processo, vou carregar aqui somente o dataset parcial final.

In [ ]:
# Carregar o novo dataset parcial
df3 = pd.read_csv('data/Leis de Sorocaba SP (Parcial 3).csv')

In [ ]:
# Concatenar os três dataframes
df = pd.concat([df1, df2, df3], ignore_index=True)

In [ ]:
# Ordenar o dataframe de modo que linhas não vazias de 'Law HTML' venham antes das vazias
df_sorted = df.sort_values(by='Law HTML', na_position='last')

# Remover duplicatas baseado na coluna 'Title_link', mantendo a primeira (não vazia, se houver)
df_unique = df_sorted.drop_duplicates(subset='Title_link', keep='first')

In [ ]:
# Verificar se existem valores faltantes na coluna 'Law HTML'
missing_mask = df_unique['Law HTML'].isna()
missing_values = missing_mask.sum()
print(f"Há {missing_values} valores faltantes na coluna 'Law HTML' por causa do CAPTCHA.")

Há 0 valores faltantes na coluna 'Law HTML' por causa do CAPTCHA.


In [ ]:
# Unir ao índice, para ver se não há leis que ficaram faltando de serem buscadas
df_index_with_content = pd.merge(df_index, df_unique, on=['Title_link'], how='left')

# Agora, vamos verificar quais linhas têm valores ausentes após o merge
missing_rows = df_index_with_content[df_index_with_content['Law HTML'].isnull() | (df_index_with_content['Law HTML'] == '')]
print(f"Há {len(missing_rows)} valores faltantes do índice na coluna 'Law HTML'.")

Há 0 valores faltantes do índice na coluna 'Law HTML'.


In [ ]:
df_index_with_content.head()

,Title_x,Title_link,Title_y,Law HTML
0,Lei Ordinária 12814/2023,https://leismunicipais.com.br/a1/sp/s/sorocaba...,Lei Ordinária 12814 2023 de Sorocaba SP,<!-- BEGIN ANCORA -->\n ...
1,Lei Ordinária 12813/2023,https://leismunicipais.com.br/a1/sp/s/sorocaba...,Lei Ordinária 12813 2023 de Sorocaba SP,<!-- BEGIN ANCORA -->\n ...
2,Lei Ordinária 12812/2023,https://leismunicipais.com.br/a1/sp/s/sorocaba...,Lei Ordinária 12812 2023 de Sorocaba SP,<!-- BEGIN ANCORA -->\n ...
3,Lei Ordinária 12811/2023,https://leismunicipais.com.br/a1/sp/s/sorocaba...,Lei Ordinária 12811 2023 de Sorocaba SP,<!-- BEGIN ANCORA -->\n ...
4,Lei Ordinária 12810/2023,https://leismunicipais.com.br/a1/sp/s/sorocaba...,Lei Ordinária 12810 2023 de Sorocaba SP,<!-- BEGIN ANCORA -->\n ...


Comparando o título obtido no índice, com o título da página de cada lei, acho que o do índice é um pouco melhor. Vamos mantê-lo no nosso dataframe final.

In [ ]:
# Removendo as colunas desnecessárias e renomeando as demais
df = df_index_with_content.drop(['Title_y'], axis=1)
df = df.rename(columns={'Title_x': 'law_title', 'Title_link': 'law_link', 'Law HTML': 'law_html'})

In [ ]:
df

,law_title,law_link,law_html
0,Lei Ordinária 12814/2023,https://leismunicipais.com.br/a1/sp/s/sorocaba...,<!-- BEGIN ANCORA -->\n ...
1,Lei Ordinária 12813/2023,https://leismunicipais.com.br/a1/sp/s/sorocaba...,<!-- BEGIN ANCORA -->\n ...
2,Lei Ordinária 12812/2023,https://leismunicipais.com.br/a1/sp/s/sorocaba...,<!-- BEGIN ANCORA -->\n ...
3,Lei Ordinária 12811/2023,https://leismunicipais.com.br/a1/sp/s/sorocaba...,<!-- BEGIN ANCORA -->\n ...
4,Lei Ordinária 12810/2023,https://leismunicipais.com.br/a1/sp/s/sorocaba...,<!-- BEGIN ANCORA -->\n ...
...,...,...,...
12807,Lei Ordinária 5/1947,https://leismunicipais.com.br/a1/sp/s/sorocaba...,<!-- BEGIN ANCORA -->\n ...
12808,Lei Ordinária 4/1947,https://leismunicipais.com.br/a1/sp/s/sorocaba...,<!-- BEGIN ANCORA -->\n ...
12809,Lei Ordinária 3/1947,https://leismunicipais.com.br/a1/sp/s/sorocaba...,<!-- BEGIN ANCORA -->\n ...
12810,Lei Ordinária 2/1947,https://leismunicipais.com.br/a1/sp/s/sorocaba...,<!-- BEGIN ANCORA -->\n ...


In [ ]:
# Salvar o dataframe final para trabalho posterior
df.to_csv('data/Leis de Sorocaba (Raw).csv', index=False)

# **Pré-processamento dos dados brutos**

Agora que já temos nosso dataset inicial, vamos começar o processo de extração de features e limpeza do HTML que foi obtido no site.

## **Limpeza do HTML**



In [8]:
# Carregar o dataset de leis
df = pd.read_csv('data/Leis de Sorocaba (Raw).csv')

In [9]:
# Visualizar o conteúdo da lei, para fazermos a limpeza
print(df.loc[0, 'law_html'])

<!-- BEGIN ANCORA -->
                                    <style>
                                        .fixar {
                                            position:fixed;
                                            margin-top: -400px !important;
                                            _margin-left: 320px;
                                            margin-left: 380px;
                                            padding-top:15px;
                                            background-color: #fff !important;
                                        }
                                        #select-art {
                                          _margin-top: 15px;
                                          width: 300px;
                                          position:absolute;
                                          display: none;
                                          margin-left: 320px;
                                        }
                                        #scro

Através deste HTML, é possível identificarmos informações úteis e muita coisa que deve ser removida. Vamos proceder com este trabalho.

In [10]:
# Remover informações sem utilidade nossa aplicação
def clean_law_html(html):
    soup = BeautifulSoup(html, 'html.parser')

    style_tag = soup.find('style')
    meuMenu_tag = soup.find(id='meuMenu')
    consolidacao_modal_tag = soup.find(id='consolidacao-modal')
    pull_right_tag = soup.find(class_='pull-right')

    if style_tag:
        style_tag.decompose()
    if meuMenu_tag:
        meuMenu_tag.decompose()
    if consolidacao_modal_tag:
        consolidacao_modal_tag.decompose()
    if pull_right_tag:
        pull_right_tag.decompose()

    for element in soup(text=lambda text: isinstance(text, Comment)):
        element.extract()

    return str(soup).strip()

In [11]:
print(clean_law_html(df.loc[0, 'law_html']))

<h2>LEI Nº 12.814, DE 26 DE MAIO DE 2023.</h2><br/><h1>(Dispõe sobre denominação de "JOSÉ ANTONIO PASCOTO" a uma via pública).</h1><br/>Projeto de Lei nº 127/2023 - autoria do Vereador DYLAN ROBERTO VIANA DANTAS.<br/><br/>A Câmara Municipal de Sorocaba decreta e eu promulgo a seguinte Lei:<br/><br/><a name="artigo_1"><span class="label label-pill label-danger">Art. 1º</span></a> Fica denominada "José Antonio Pascoto" o trecho da Rua Barão VL R/16, com início na Rua Benedito Clemente de Souza e término em Cul-de-Sac localizada no bairro Vila Barão, nesta cidade.<br/><br/><a name="artigo_2"><span class="label label-pill label-danger">Art. 2º</span></a> A placa indicativa conterá, além do nome, a expressão "Cidadão Emérito - 1950 - 2010".<br/><br/><a name="artigo_3"><span class="label label-pill label-danger">Art. 3º</span></a> As despesas decorrentes da execução da presente Lei correrão por conta de verba orçamentária própria.<br/><br/><a name="artigo_4"><span class="label label-pill lab

## **Extração de informações**

In [12]:
# Extrair informações relevantes
def get_law_info(html):
    soup = BeautifulSoup(html, 'html.parser')
    summary_tag = soup.find('h1') # Ementa da Lei
    title_tag = soup.find('h2') # Título da Lei
    summary = summary_tag.get_text() if summary_tag else None
    title = title_tag.get_text() if title_tag else None
    for tag in soup.find_all('br'):
        tag.replace_with('\n')
    full_text = soup.get_text().strip()
    return {
        'title': title,
        'title_tag': str(title_tag) if title_tag else None,
        'summary': summary,
        'summary_tag': str(summary_tag) if summary_tag else None,
        'full_text': full_text,
        'full_html': html
        }

In [13]:
# Exibir o conteúdo extraído
clean_html = clean_law_html(df.loc[0, 'law_html'])
law_info = get_law_info(clean_html)
print(json.dumps(law_info, indent=4, ensure_ascii=False))

{
    "title": "LEI Nº 12.814, DE 26 DE MAIO DE 2023.",
    "title_tag": "<h2>LEI Nº 12.814, DE 26 DE MAIO DE 2023.</h2>",
    "summary": "(Dispõe sobre denominação de \"JOSÉ ANTONIO PASCOTO\" a uma via pública).",
    "summary_tag": "<h1>(Dispõe sobre denominação de \"JOSÉ ANTONIO PASCOTO\" a uma via pública).</h1>",
    "full_text": "LEI Nº 12.814, DE 26 DE MAIO DE 2023.\n(Dispõe sobre denominação de \"JOSÉ ANTONIO PASCOTO\" a uma via pública).\nProjeto de Lei nº 127/2023 - autoria do Vereador DYLAN ROBERTO VIANA DANTAS.\n\nA Câmara Municipal de Sorocaba decreta e eu promulgo a seguinte Lei:\n\nArt. 1º Fica denominada \"José Antonio Pascoto\" o trecho da Rua Barão VL R/16, com início na Rua Benedito Clemente de Souza e término em Cul-de-Sac localizada no bairro Vila Barão, nesta cidade.\n\nArt. 2º A placa indicativa conterá, além do nome, a expressão \"Cidadão Emérito - 1950 - 2010\".\n\nArt. 3º As despesas decorrentes da execução da presente Lei correrão por conta de verba orçamentá

In [14]:
# Exibir somente o texto final
print(law_info['full_text'])

LEI Nº 12.814, DE 26 DE MAIO DE 2023.
(Dispõe sobre denominação de "JOSÉ ANTONIO PASCOTO" a uma via pública).
Projeto de Lei nº 127/2023 - autoria do Vereador DYLAN ROBERTO VIANA DANTAS.

A Câmara Municipal de Sorocaba decreta e eu promulgo a seguinte Lei:

Art. 1º Fica denominada "José Antonio Pascoto" o trecho da Rua Barão VL R/16, com início na Rua Benedito Clemente de Souza e término em Cul-de-Sac localizada no bairro Vila Barão, nesta cidade.

Art. 2º A placa indicativa conterá, além do nome, a expressão "Cidadão Emérito - 1950 - 2010".

Art. 3º As despesas decorrentes da execução da presente Lei correrão por conta de verba orçamentária própria.

Art. 4º Esta Lei entra em vigor na data de sua publicação.

Palácio dos Tropeiros "Dr. José Theodoro Mendes", em 26 de maio de 2 023, 368º da Fundação de Sorocaba.

RODRIGO MAGANHATO
Prefeito Municipal

DOUGLAS DOMINGOS DE MORAES
Secretário Jurídico

JOÃO ALBERTO CORRÊA MAIA
Secretário de Governo

GLAUCO ENRICO BERNARDES FOGAÇA
Secretário

Aparentemente nossa extração de dados funcionou. Vamos então aplicar estas funções para todas as observações do nosso dataset.

In [15]:
%%time
# Extrair conteúdo de todas as leis
df_law_info = df['law_html'].apply(clean_law_html).apply(get_law_info).apply(pd.Series)

CPU times: user 52 s, sys: 0 ns, total: 52 s
Wall time: 52 s


In [16]:
# Visualizar extração de dados
df_law_info

,title,title_tag,summary,summary_tag,full_text,full_html
0,"LEI Nº 12.814, DE 26 DE MAIO DE 2023.","<h2>LEI Nº 12.814, DE 26 DE MAIO DE 2023.</h2>","(Dispõe sobre denominação de ""JOSÉ ANTONIO PAS...","<h1>(Dispõe sobre denominação de ""JOSÉ ANTONIO...","LEI Nº 12.814, DE 26 DE MAIO DE 2023.\n(Dispõe...","<h2>LEI Nº 12.814, DE 26 DE MAIO DE 2023.</h2>..."
1,"LEI Nº 12.813, DE 26 DE MAIO DE 2023.","<h2>LEI Nº 12.813, DE 26 DE MAIO DE 2023.</h2>","(Dispõe sobre denominação de ""Vitor Hage"" a um...","<h1>(Dispõe sobre denominação de ""Vitor Hage"" ...","LEI Nº 12.813, DE 26 DE MAIO DE 2023.\n(Dispõe...","<h2>LEI Nº 12.813, DE 26 DE MAIO DE 2023.</h2>..."
2,"LEI Nº 12.812, DE 26 DE MAIO DE 2023.","<h2>LEI Nº 12.812, DE 26 DE MAIO DE 2023.</h2>","(Dispõe sobre denominação de ""LISARDO CUNHA DI...","<h1>(Dispõe sobre denominação de ""LISARDO CUNH...","LEI Nº 12.812, DE 26 DE MAIO DE 2023.\n(Dispõe...","<h2>LEI Nº 12.812, DE 26 DE MAIO DE 2023.</h2>..."
3,"LEI Nº 12.811, DE 26 DE MAIO DE 2023.","<h2>LEI Nº 12.811, DE 26 DE MAIO DE 2023.</h2>","(Dispõe sobre denominação de ""Desirre Ferraz C...","<h1>(Dispõe sobre denominação de ""Desirre Ferr...","LEI Nº 12.811, DE 26 DE MAIO DE 2023.\n(Dispõe...","<h2>LEI Nº 12.811, DE 26 DE MAIO DE 2023.</h2>..."
4,"LEI Nº 12.810, DE 26 DE MAIO DE 2023.","<h2>LEI Nº 12.810, DE 26 DE MAIO DE 2023.</h2>","(Institui, no calendário oficial do Município ...","<h1>(Institui, no calendário oficial do Municí...","LEI Nº 12.810, DE 26 DE MAIO DE 2023.\n(Instit...","<h2>LEI Nº 12.810, DE 26 DE MAIO DE 2023.</h2>..."
...,...,...,...,...,...,...
12807,"LEI Nº 5, DE 1º DE OUTUBRO DE 1.947","<h2>LEI Nº 5, DE 1º DE OUTUBRO DE 1.947</h2>",DISPÕE SÔBRE ABERTURA DE CRÉDITO ESPECIAL DE C...,<h1>DISPÕE SÔBRE ABERTURA DE CRÉDITO ESPECIAL ...,"LEI Nº 5, DE 1º DE OUTUBRO DE 1.947(Revogada p...","<h2>LEI Nº 5, DE 1º DE OUTUBRO DE 1.947</h2><b..."
12808,"LEI Nº 4, DE 1º DE OUTUBRO DE 1.947","<h2>LEI Nº 4, DE 1º DE OUTUBRO DE 1.947</h2>",DISPÕE SÔBRE CONCESSÃO DE AUXÍLIO E DÁ OUTRAS ...,<h1>DISPÕE SÔBRE CONCESSÃO DE AUXÍLIO E DÁ OUT...,"LEI Nº 4, DE 1º DE OUTUBRO DE 1.947(Revogada p...","<h2>LEI Nº 4, DE 1º DE OUTUBRO DE 1.947</h2><b..."
12809,"LEI Nº 3, DE 19 DE SETEMBRO DE 1.947","<h2>LEI Nº 3, DE 19 DE SETEMBRO DE 1.947</h2>",DISPÕE SÔBRE CONCESSÃO DE LICENÇA PRÊMIO AOS F...,<h1>DISPÕE SÔBRE CONCESSÃO DE LICENÇA PRÊMIO A...,"LEI Nº 3, DE 19 DE SETEMBRO DE 1.947\nDISPÕE S...","<h2>LEI Nº 3, DE 19 DE SETEMBRO DE 1.947</h2><..."
12810,"LEI Nº 2, DE 17 DE SETEMBRO DE 1.947","<h2>LEI Nº 2, DE 17 DE SETEMBRO DE 1.947</h2>",DISPÕE SÔBRE APREENSÃO E ELIMINAÇÃO DE ANIMAIS,<h1>DISPÕE SÔBRE APREENSÃO E ELIMINAÇÃO DE ANI...,"LEI Nº 2, DE 17 DE SETEMBRO DE 1.947(Revogada ...","<h2>LEI Nº 2, DE 17 DE SETEMBRO DE 1.947</h2><..."


In [ ]:
# Salvar o dataset com as informações extraídas
df_law_info.to_csv('data/Leis de Sorocaba.csv', index=False)

## **Busca por inconsistências**

Vamos agora verificar se a extração falhou em alguma das leis.

Caso positivo, pode ser que o HTML tenha uma estrutura diferente para cada lei, não sendo padronizado.

In [ ]:
# Carregar o dataset mais atual
df_law_info = pd.read_csv('data/Leis de Sorocaba.csv')

In [ ]:
incorrect_extractions = df_law_info.isna().any(axis=1)
print(f'Existem {incorrect_extractions.sum()} observações onde a extração não foi completa')

Existem 35 observações onde a extração não foi completa


In [ ]:
# Exibir observações que possam ter dados faltantes
df_law_info[incorrect_extractions]

,title,title_tag,summary,summary_tag,full_text,full_html
783,"LEI Nº 12.028, DE 24 DE JUNHO DE 2019","<h2>LEI Nº 12.028, DE 24 DE JUNHO DE 2019</h2>",NaN,NaN,"LEI Nº 12.028, DE 24 DE JUNHO DE 2019Lei Munic...","<h2>LEI Nº 12.028, DE 24 DE JUNHO DE 2019</h2>..."
1001,"LEI Nº 11.810, DE 9 DE OUTUBRO DE 2018","<h2>LEI Nº 11.810, DE 9 DE OUTUBRO DE 2018</h2>",NaN,NaN,"LEI Nº 11.810, DE 9 DE OUTUBRO DE 2018 \n\nDis...","<h2>LEI Nº 11.810, DE 9 DE OUTUBRO DE 2018</h2..."
1074,"LEI Nº 11.737, DE 29 DE JUNHO DE 2018","<h2>LEI Nº 11.737, DE 29 DE JUNHO DE 2018</h2>",NaN,NaN,"LEI Nº 11.737, DE 29 DE JUNHO DE 2018 \n\nDisp...","<h2>LEI Nº 11.737, DE 29 DE JUNHO DE 2018</h2>..."
1112,"LEI Nº 11.699, DE 16 DE ABRIL DE 2018","<h2>LEI Nº 11.699, DE 16 DE ABRIL DE 2018</h2>",NaN,NaN,"LEI Nº 11.699, DE 16 DE ABRIL DE 2018Lei nº 11...","<h2>LEI Nº 11.699, DE 16 DE ABRIL DE 2018</h2>..."
1895,"LEI Nº 10.916, DE 30 DE JULHO DE 2014.","<h2>LEI Nº 10.916, DE 30 DE JULHO DE 2014.</h2>",NaN,NaN,"LEI Nº 10.916, DE 30 DE JULHO DE 2014.\n\nAUTO...","<h2>LEI Nº 10.916, DE 30 DE JULHO DE 2014.</h2..."
2158,LEI Nº 10.654/2013.,<h2>LEI Nº 10.654/2013.</h2>,NaN,NaN,"LEI Nº 10.654/2013.\n\n(ATO INEXISTENTE, NUMER...",<h2>LEI Nº 10.654/2013.</h2><br/><br/>(ATO INE...
2681,"LEI Nº 10.131, DE 30 DE MAIO DE 2012. (Suspens...","<h2>LEI Nº 10.131, DE 30 DE MAIO DE 2012. <b>(...",NaN,NaN,"LEI Nº 10.131, DE 30 DE MAIO DE 2012. (Suspens...","<h2>LEI Nº 10.131, DE 30 DE MAIO DE 2012. <b>(..."
2770,"LEI Nº 10.042, DE 25 DE ABRIL DE 2012","<h2>LEI Nº 10.042, DE 25 DE ABRIL DE 2012</h2>",NaN,NaN,"LEI Nº 10.042, DE 25 DE ABRIL DE 2012(Regulame...","<h2>LEI Nº 10.042, DE 25 DE ABRIL DE 2012</h2>..."
3623,"LEI Nº 9189, DE 22 DE JUNHO DE 2010.","<h2>LEI Nº 9189, DE 22 DE JUNHO DE 2010.</h2>",NaN,NaN,"LEI Nº 9189, DE 22 DE JUNHO DE 2010.\nAUTORIZA...","<h2>LEI Nº 9189, DE 22 DE JUNHO DE 2010.</h2><..."
4525,"LEI Nº 8287, DE 22 DE OUTUBRO DE 2007.","<h2>LEI Nº 8287, DE 22 DE OUTUBRO DE 2007.</h2>",NaN,NaN,"LEI Nº 8287, DE 22 DE OUTUBRO DE 2007.\n\nDISP...","<h2>LEI Nº 8287, DE 22 DE OUTUBRO DE 2007.</h2..."


Vamos analisar manualmente a estrutura HTML onde a extração do resumo da lei falhou:

In [ ]:
links_df = df.loc[incorrect_extractions, ['law_title', 'law_link']]

# Função para tornar um link clicável
def make_clickable(link):
    return f'<a target="_blank" href="{link}">{link}</a>'

# Aplicar a função para tornar links clicáveis
links_df.style.format({'law_link': make_clickable})

,law_title,law_link
783,Lei Ordinária 12028/2019,https://leismunicipais.com.br/a1/sp/s/sorocaba/lei-ordinaria/2019/1203/12028/lei-ordinaria-n-12028-2019-altera-o-art-4-lei-n-4812-de-12-de-maio-de-1995-que-disciplina-a-protecao-o-corte-e-a-poda-de-vegetacao-de-porte-arboreo-e-da-outras-providencias
1001,Lei Ordinária 11810/2018,https://leismunicipais.com.br/a1/sp/s/sorocaba/lei-ordinaria/2018/1181/11810/lei-ordinaria-n-11810-2018-dispoe-sobre-regras-especificas-a-serem-observadas-no-projeto-no-licenciamento-na-execucao-na-manutencao-e-na-utilizacao-de-conteineres-como-residencias-ou-estabelecimentos-comerciais-de-qualquer-natureza-e-da-outras-providencias
1074,Lei Ordinária 11737/2018,https://leismunicipais.com.br/a1/sp/s/sorocaba/lei-ordinaria/2018/1174/11737/lei-ordinaria-n-11737-2018-dispoe-sobre-eficaz-acesso-as-informacoes-referentes-aos-pontos-de-venda-credenciados-do-cartao-horario-da-zona-azul-estacionamento-rotativo-obrigatorio-e-da-outras-providencias
1112,Lei Ordinária 11699/2018,https://leismunicipais.com.br/a1/sp/s/sorocaba/lei-ordinaria/2018/1170/11699/lei-ordinaria-n-11699-2018-dispoe-sobre-a-obrigatoriedade-de-instalacao-de-placas-de-metal-escritas-em-braile-nos-pontos-de-onibus-no-municipio-de-sorocaba-e-da-outras-providencias
1895,Lei Ordinária 10916/2014,https://leismunicipais.com.br/a1/sp/s/sorocaba/lei-ordinaria/2014/1092/10916/lei-ordinaria-n-10916-2014-autoriza-o-poder-executivo-a-contratar-operacao-de-credito-internacional-com-o-banco-de-desenvolvimento-da-america-latina-caf-a-oferecer-garantias-e-da-outras-providencias
2158,Lei Ordinária 10654/2013,https://leismunicipais.com.br/a1/sp/s/sorocaba/lei-ordinaria/2013/1066/10654/lei-ordinaria-n-10654-2013-ato-inexistente-numeracao-nao-utilizada
2681,Lei Ordinária 10131/2012,https://leismunicipais.com.br/a1/sp/s/sorocaba/lei-ordinaria/2012/1014/10131/lei-ordinaria-n-10131-2012-dispoe-sobre-a-obrigatoriedade-do-fornecimento-gratuito-de-sacolas-plasticas-oxibiodegradaveis-obp-s-ou-retornaveis-aos-respectivos-consumidores-pelos-estabelecimentos-que-menciona
2770,Lei Ordinária 10042/2012,https://leismunicipais.com.br/a1/sp/s/sorocaba/lei-ordinaria/2012/1005/10042/lei-ordinaria-n-10042-2012-dispoe-sobre-a-isencao-de-pagamento-de-taxa-de-inscricao-em-concursos-publicos-no-ambito-municipal-nos-casos-que-especifica-e-da-outras-providencias
3623,Lei Ordinária 9189/2010,"<a target=""_blank"" href=""https://leismunicipais.com.br/a1/sp/s/sorocaba/lei-ordinaria/2010/919/9189/lei-ordinaria-n-9189-2010-autoriza-o-municipio-de-sorocaba-a-celebrar-convenio-com-o-governo-do-estado-de-sao-paulo-atraves-da-secretaria-de-economia-e-planejamento-esta-por-meio-de-sua-unidade-de-articulacao-com-municipios-visando-o-recebimento-de-recursos-financeiros-provenientes-de-emenda-parlamentar-para-pavimentacao-da-rua-john-boyd-dunlop-no-bairro-iporanga-e-de-ruas-do-bairro-mineirao-e-do-residencial-conjunto-sao-joaquim-vila-barao-e-da-outras-providencias"">https://leismunicipais.com.br/a1/sp/s/sorocaba/lei-ordinaria/2010/919/9189/lei-ordinaria-n-9189-2010-autoriza-o-municipio-de-sorocaba-a-celebrar-convenio-com-o-governo-do-estado-de-sao-paulo-atraves-da-secretaria-de-economia-e-planejamento-esta-por-meio-de-sua-unidade-de-articulacao-com-municipios-visando-o-recebimento-de-recursos-financeiros-provenientes-de-emenda-parlamentar-para-pavimentacao-da-rua-john-boyd-dunlop-no-bairro-iporanga-e-de-ruas-do-bairro-mineirao-e-do-residencial-conjunto-sao-joaquim-vila-barao-e-da-outras-providencias"
4525,Lei Ordinária 8287/2007,https://leismunicipais.com.br/a1/sp/s/sorocaba/lei-ordinaria/2007/829/8287/lei-ordinaria-n-8287-2007-dispoe-sobre-a-obrigacao-dos-estabelecimentos-de-ensino-municipais-em-manterem-em-sua-merenda-alimentacao-diferenciada-e-adequada-aos-alunos-portadores-de-diabetes-e-da-outras-providencias


De maneira geral, pelo o que pudemos observar, estas leis não foram inseridas corretamente na base de dados.

Existem casos onde a ementa da lei sofreu alterações. Porém, não sabemos se isso afeta a padronização do HTML. Provavelmente existem outros casos de alteração em nosso dataset e que a padronização seguiu corretamente. Vamos verificar.

As leis que sofreram alterações possuem a tag `<s>`. Se existirem ementas de lei onde existe essa tag, a padronização funciona corretamente.

In [ ]:
# Regex para buscar títulos onde existe a tag <s> dentro de <h1>
regex = r'<s>.*</h1>'

# Filtrar onde a coluna 'summary_tag' contém o regex
mask = df_law_info['summary_tag'].str.contains(regex, regex=True, na=False)

df_filtered = df_law_info[['summary_tag']][mask]

df_filtered.head()

,summary_tag
439,<h1><s>(Dispõe sobre o credenciamento de admin...
656,<h1><s>(Cria a Patrulha Ambiental e institui a...
1260,<h1><s>Obriga a Prefeitura Municipal de Soroca...
1274,<h1><s>Obriga a Prefeitura Municipal de Soroca...
1748,<h1>DISPÕE SOBRE A RECLASSIFICAÇÃO DOS VENCIME...


In [ ]:
# Exibir o código HTML da tag <h1>
html = df_filtered.loc[439, 'summary_tag']
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify())

<h1>
 <s>
  (Dispõe sobre o credenciamento de administradoras de planos de saúde aos servidores contratados sob o regime da Consolidação das Leis do Trabalho - CLT e seus dependentes e dá outras providências).
 </s>
 <b>
  Dispõe sobre o credenciamento de administradoras e operadoras de planos de saúde aos servidores contratados sob o regime da Consolidação das Leis do Trabalho - CLT e Conselheiros Tutelares e seus dependentes e dá outras providência. (Redação dada pela Lei nº
  <a class="link_law" data-id="7711952" data-original-title=" Data da Norma: 08.04.2022 - (Altera a ementa e a redação do artigo 1º, da Lei nº 12.373, de 20 de setembro de 2021 que dispõe sobre o credenciamento de administradoras de planos de saúde aos servidores contratados sob o regime da Consolidação das Leis do Trabalho - CLT e seus dependentes e dá outras providências)." data-toggle="tooltip" href="https://leismunicipais.com.br/a1/sp/s/sorocaba/lei-ordinaria/2022/1253/12536/lei-ordinaria-n-12536-2022-altera-

In [ ]:
# Renderizar o código HTML da tag <h1>
display(HTML(html))

Está confirmado que as alterações na ementa da lei não alteram a padronização do HTML.

Então, os casos de extração incorreta são todos por problemas de inserção na base de dados do site.

Como são pouquíssimas observações, vamos ignorar estes casos se precisarmos trabalhar somente com as ementas.

 Talvez seja interessante remover os trechos de lei alterados, mas podemos verificar isso depois.